In [1]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

import matplotlib.image as mpimg
from keras import backend as K

import skimage.io
import matplotlib.pyplot as plt
import cv2
import glob
from skimage.transform import resize
from keras.applications.xception import preprocess_input, decode_predictions
import numpy as np
from tqdm import tqdm

import keras
from keras.applications.xception import Xception

from keras.utils import to_categorical

%matplotlib inline
K.clear_session()

In [2]:
model = Xception(weights='imagenet')
model.compile(loss=keras.losses.categorical_crossentropy, optimizer= keras.optimizers.SGD(lr=0.01),metrics = ['accuracy'])

In [3]:
num_classes=1000

file = open("drive/My Drive/Tesi/Colab Notebooks/ILSVRC2014_clsloc_validation_ground_truth.txt","r")
label = file.readlines()
file.close()
label =[int(item) for item in label]


In [4]:
import scipy.io
meta = scipy.io.loadmat("drive/My Drive/Tesi/Colab Notebooks/meta_clsloc.mat")
original_idx_to_synset = {}
synset_to_name = {}
for i in range(1000):
    ilsvrc2014_id = int(meta["synsets"][0,i][0][0][0])
    synset = meta["synsets"][0,i][1][0]
    name = meta["synsets"][0,i][2][0]
    original_idx_to_synset[ilsvrc2014_id] = synset
    synset_to_name[synset] = name
synset_to_keras_idx = {}
keras_idx_to_name = {}
f = open("drive/My Drive/Tesi/Colab Notebooks/synset_words.txt","r")
idx = 0
for line in f:
    parts = line.split(" ")
    synset_to_keras_idx[parts[0]] = idx
    keras_idx_to_name[idx] = " ".join(parts[1:])
    idx += 1
f.close()
def convert_original_idx_to_keras_idx(idx):
    return synset_to_keras_idx[original_idx_to_synset[idx]]

In [5]:
label = np.array([convert_original_idx_to_keras_idx(idx) for idx in label])
label = to_categorical(label , num_classes)

In [ ]:
pbar = tqdm(total=50000)
img_rows=299; img_cols=299; img_channels=3;
x_val = []
y_val = []
test_accuracy_list = []
test_loss_list = []
j=1
i=1
k=0
for filename in sorted(glob.glob("drive/My Drive/Tesi/Colab Notebooks/ILSVRC2014_images_val/*.JPEG")):
    img = np.float32(skimage.io.imread(filename))
    img = resize(img, (img_rows, img_cols), order=1)
    if img.shape!=(299,299,3):
      img = np.stack((img,img,img), -1)
    #img = np.reshape(img, (1,img.shape[0],img.shape[1],img.shape[2]))
    img = preprocess_input(img)

    x_val.append(img)
    if i==2500:
      #for item in x_val:
        #if item.shape!=(299,299,3):
          #print("Shape prima:",item.shape)
          #item = np.stack((item,item,item), -1)
          #print("Shape dopo:",item.shape)
      #print("Dimensione x_val:",len(x_val))

      x_val = np.reshape(x_val , (2500,img_rows,img_cols,img_channels))
      #x_val = np.float32(x_val) / 255
      y_val=label[k:(k+2500)]
      #for item in x_val:
          #plt.figure(); plt.imshow(item/255); plt.show();
    
      #print("Dimensione x_val:",len(x_val))
      #print("Dimensione y_val:",len(y_val))
      test_loss, test_accuracy=model.evaluate(x_val, y_val, verbose=True)
      test_loss_list.append(test_loss)
      test_accuracy_list.append(test_accuracy)
      print('Test loss:',test_loss)
      print('Test accuracy:',test_accuracy)
      x_val = []
      y_val = []
      i=0
      j=j+1
      k=k+2500
    
    i=i+1
    pbar.update()

pbar.close()
accuracy_media=sum(test_accuracy_list) / float(len(test_accuracy_list)) 
print("Accuracy media:",accuracy_media)



  5%|▍         | 2499/50000 [26:37<7:30:21,  1.76it/s]

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


  5%|▌         | 2500/50000 [26:58<88:34:54,  6.71s/it]

Test loss: 0.929078028011322
Test accuracy: 0.79


 10%|█         | 5000/50000 [53:51<64:58:28,  5.20s/it]

...

 75%|███████▌  | 37500/50000 [6:45:47<16:20:59,  4.71s/it]

Test loss: 0.9030077569961548
Test accuracy: 0.7912


 80%|████████  | 40000/50000 [7:15:56<13:20:36,  4.80s/it]

Test loss: 0.942937330532074
Test accuracy: 0.7876


 83%|████████▎ | 41543/50000 [7:34:21<1:49:13,  1.29it/s]

In [ ]:
#img_rows=299; img_cols=299; img_channels=3;
#i=1
#for filename in sorted(glob.glob("drive/My Drive/Tesi/Colab Notebooks/ILSVRC2014_images_val/*.JPEG")):
 #   img = np.float32(skimage.io.imread(filename))
    #plt.figure(); plt.imshow(img/255); plt.show();
  #  img = resize(img, (img_rows, img_cols), order=1)
   # img = np.reshape(img, (1,img.shape[0],img.shape[1],img.shape[2]))
    #img = preprocess_input(img)
    #preds = model.predict(img)
    #preds_d = decode_predictions(preds,top=3)
    #for p in preds_d:
     #   print(p) 
    #indici = np.argmax(preds,-1)
    #for index in indici:
     # print(index)
    #if i==10:
     # break
    #i=i+1